In [5]:
import pandas as pd
df=pd.read_csv('D:/Users/bdhooda/Desktop/Sunil_Denali/books/Data/sentiment_data.csv')

In [3]:
df.columns = ['review', 'sentiment']

In [3]:
import numpy as np
np.random.seed(0)
#randomize data frame
df = df.reindex(np.random.permutation(df.index))
#save data frame to a csv file called movie_data.csv
df.to_csv('movie_data.csv', index = False, encoding = 'utf-8')

## ==== Read CSV file and check first 5 rows ==== ###
df = pd.read_csv('D:/Users/bdhooda/Desktop/Sunil_Denali/books/Data/sentiment_data.csv', encoding = 'utf-8')
df.head()

review  sentiment
0  In 1974, the teenager Martha Moxley (Maggie Gr...          1
1  OK... so... I really like Kris Kristofferson a...          0
2  ***SPOILER*** Do not read this, if you think a...          0
3  hi for all the people who have seen this wonde...          1
4  I recently bought the DVD, forgetting just how...          0

In [4]:
df.loc[0, 'review'][-999:]

'n trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich family used their influence to cover the murder for more than twenty years. However, a snoopy detective and convicted perjurer in disgrace was able to disclose how the hideous crime was committed. The screenplay shows the investigation of Mark and the last days of Martha in parallel, but there is a lack of the emotion in the dramatization. My vote is seven.<br /><br />Title (Brazil): Not Availabl

In [5]:
def tokenizer1(text):
    return text.split()
  
tokenizer1('running like running and thus they run')

['running', 'like', 'running', 'and', 'thus', 'they', 'run']

In [6]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('running like running and thus they run')

['run', 'like', 'run', 'and', 'thu', 'they', 'run']

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

stop = stopwords.words('english')
[w for w in tokenizer_porter('a running likes running and runs a lot')[-10:] if w not in stop]

[nltk_data] Error loading stopwords: HTTP Error 401: Authorization
[nltk_data]     Required


['run', 'like', 'run', 'run', 'lot']

In [13]:
import numpy as np
import re
from nltk.corpus import stopwords

from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)    
doc_stream=stream_docs(path='D:/Users/bdhooda/Desktop/Sunil_Denali/books/Data/sentiment_data.csv')

stop = stopwords.words('english')

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label
            
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y
 
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
      ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized
  
vect = HashingVectorizer(decode_error='ignore', 
                       n_features=2**21,
                       preprocessor=None, 
                       tokenizer=tokenizer)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)

NameError: name 'stream_docs' is not defined

In [10]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

NameError: name 'get_minibatch' is not defined

In [11]:
import pickle
import os 
dest = os.path.join('movie', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop,open(os.path.join(dest,'stopwords.pkl'),'wb'),protocol = 4)
pickle.dump(clf,open(os.path.join(dest,'classifier.pkl'),'wb'),protocol=4)